In [1]:
import pandas as pd
from IPython.display import display
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import mannwhitneyu
import gc
import numpy as np

In [2]:
# df = pd.read_csv("c:\\data\\e-commerce\\2019-Oct.csv", usecols=['event_time', 'event_type', 'product_id', 'brand', 'category_code', 'price', 'user_id', 'user_session'])
# df.head(2)

In [3]:
# df['category_code'] = df['category_code'].astype('category')
# df['category_code'] = df['category_code'].cat.add_categories('unknown.')
# df['category_code'].fillna('unknown.', inplace=True)

# df['general_category'] = df['category_code'].apply(lambda x: x.split('.')[0])
# display(df.head(2))

In [4]:
# df['user_session'] = df['user_session'].astype('category').cat.codes
# df.head()

In [5]:
# df.head(2)

In [6]:
# # datetime 처리
# df['event_time'] = pd.to_datetime(df['event_time'])
# df['date'] = df['event_time'].dt.date

# # 사용자별, 시간순 정렬
# df = df.sort_values(['user_id', 'event_time'])

# # 구매 발생 플래그
# df['is_purchase'] = df['event_type'] == 'purchase'

# # 하루 단위 기준 세션 초기화
# df['daily_session'] = df['user_id'].astype(str) + '_' + df['date'].astype(str)

# # 구매 발생 시 세션 쪼개기: 구매 이후는 새로운 세션으로
# def split_on_purchase(group):
#     session_id = []
#     counter = 0
#     for is_p in group['is_purchase']:
#         session_id.append(counter)
#         if is_p:
#             counter += 1
#     return session_id

# # 사용자 + 날짜별로 쪼개서 세션 나누기
# df['session_split_id'] = df.groupby(['user_id', 'date']).apply(split_on_purchase).explode().values

# # reallyreal진짜최종의최총의최최종
# df['custom_session'] = (
#     df['user_id'].astype(str) + '_' +
#     df['date'].astype(str) + '_' +
#     df['session_split_id'].astype(str)
# )
# df.head(2)


In [7]:
# df.to_csv("c:\\data\\e-commerce\\custom_session.csv", index=False) # event_time, event_type, product_id, category_code, brand, price, user_id, user_session, general_category, buy, date, is_purchase, daily_session, session_split_id, custom_session
custom_session = pd.read_csv("c:\\data\\e-commerce\\custom_session.csv", usecols=['event_time', 'event_type', 'product_id', 'category_code', 'brand', 'price', 'user_id', 'general_category', 'buy', 'custom_session']);custom_session.head(2)
custom_session.head(2)

,event_time,event_type,product_id,category_code,brand,price,user_id,general_category,buy,custom_session
0,2019-10-23 20:04:08+00:00,view,7002639,kids.carriage,bumbleride,769.65,33869381,kids,0,33869381_2019-10-23_0
1,2019-10-13 00:13:46+00:00,view,10600284,unknown.,NaN,0.00,64078358,unknown,0,64078358_2019-10-13_0


In [8]:
buy_sessions = custom_session[custom_session['event_type'] == 'purchase']['custom_session'].unique()

buy_log = custom_session[custom_session['custom_session'].isin(buy_sessions)]
not_buy_log = custom_session[~custom_session['custom_session'].isin(buy_sessions)]
custom_session['buy'] = custom_session['custom_session'].isin(buy_log['custom_session']).astype(int)
custom_session.head(2)

,event_time,event_type,product_id,category_code,brand,price,user_id,general_category,buy,custom_session
0,2019-10-23 20:04:08+00:00,view,7002639,kids.carriage,bumbleride,769.65,33869381,kids,0,33869381_2019-10-23_0
1,2019-10-13 00:13:46+00:00,view,10600284,unknown.,NaN,0.00,64078358,unknown,0,64078358_2019-10-13_0


In [9]:
median_price_category = custom_session.groupby('general_category')['price'].median().to_dict()
custom_session['relative_price'] = custom_session['price'] / custom_session['general_category'].map(median_price_category)
custom_session.head(2)

,event_time,event_type,product_id,category_code,brand,price,user_id,general_category,buy,custom_session,relative_price
0,2019-10-23 20:04:08+00:00,view,7002639,kids.carriage,bumbleride,769.65,33869381,kids,0,33869381_2019-10-23_0,6.986022
1,2019-10-13 00:13:46+00:00,view,10600284,unknown.,NaN,0.00,64078358,unknown,0,64078358_2019-10-13_0,0.000000


In [10]:
custom_session['custom_session'] = custom_session['custom_session'].astype('category').cat.codes
custom_session.head(2)

,event_time,event_type,product_id,category_code,brand,price,user_id,general_category,buy,custom_session,relative_price
0,2019-10-23 20:04:08+00:00,view,7002639,kids.carriage,bumbleride,769.65,33869381,kids,0,431,6.986022
1,2019-10-13 00:13:46+00:00,view,10600284,unknown.,NaN,0.00,64078358,unknown,0,42508,0.000000


In [ ]:
def log_dictionary(df):

    df = df.sort_values(['user_id', 'event_time'])
    df['event_time'] = pd.to_datetime(df['event_time'])

    session_data = []

    for session, group in df.groupby('custom_session'):
        group = group.sort_values('event_time')

        times = group['event_time'].tolist()
        time_since_start = [(t - times[0]).total_seconds() for t in times]
        time_since_prev = [0] + [(times[i] - times[i-1]).total_seconds() for i in range(1, len(times))]

        events = []
        for _, row in group.iterrows():
            if row['event_type'] == 'purchase':
                continue  # 구매 제외
            
            events.append({
                "event_type": row["event_type"],
                "product_id": row["product_id"],
                "category_code": row["category_code"] if pd.notna(row["category_code"]) else "unknown.",
                "general_category": row["general_category"] if pd.notna(row["general_category"]) else "unknown",
                "brand": row["brand"] if pd.notna(row["brand"]) else "unknown",
                "price": row["price"],
                "relative_price": row["relative_price"],
                "time_since_start": time_since_start[len(events)],
                "time_since_prev": time_since_prev[len(events)]
            })

        session_data.append({
            "session": session,
            "event": events,
            "buy": int(group["buy"].max())
        })

    return pd.DataFrame(session_data)

In [12]:
log_dict = log_dictionary(custom_session)

In [13]:
# pd.set_option("display.max_colwidth", None)
# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_columns", None) 

pd.reset_option("display.max_colwidth")
pd.reset_option("display.max_rows")
pd.reset_option("display.max_columns")


In [14]:
display(log_dict.head(2))

,session,event,buy
0,0,"[{'event_type': 'view', 'product_id': 22200103...",0
1,1,"[{'event_type': 'view', 'product_id': 6902133,...",0


In [15]:
from sklearn.preprocessing import LabelEncoder
le_event = LabelEncoder()
le_event.fit(custom_session['event_type'].unique())
le_event.classes_

array(['cart', 'purchase', 'view'], dtype=object)

In [16]:
def encode_event_type(events):
    for event in events:
        event['event_type'] = le_event.transform([event['event_type']])[0]
    return events
# 적용
log_dict['event'] = log_dict['event'].apply(encode_event_type)

In [17]:
log_dict.head(2)

,session,event,buy
0,0,"[{'event_type': 2, 'product_id': 22200103, 'ca...",0
1,1,"[{'event_type': 2, 'product_id': 6902133, 'cat...",0


In [18]:
df = log_dict.copy()

In [ ]:
# 전체 파이프라인: 데이터 파싱 -> vocabs -> tf.data -> Transformer 모델 -> 학습
import ast
import json
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [20]:
# 1) 파싱
# columns = ['session', 'event', 'buy']
# event 컬럼은 문자열화된 리스트(예: "[{...}, {...}]")
# parse if string
def parse_events(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except Exception:
            try:
                return json.loads(x)
            except Exception:
                return []
    elif isinstance(x, list):
        return x
    else:
        return []

df['events_parsed'] = df['event'].apply(parse_events)

# quick inspect
print("n_sessions:", len(df))
print("example session events:", df['events_parsed'].iloc[0])

n_sessions: 42509
example session events: [{'event_type': np.int64(2), 'product_id': 22200103, 'category_code': 'unknown.', 'general_category': 'unknown', 'brand': 'unknown', 'price': 15.77, 'relative_price': 0.17504717504717504, 'time_since_start': 0.0, 'time_since_prev': 0}]


In [21]:
# 2) Extract features per event and build vocabs for categorical fields
cat_fields = ['event_type', 'category_code', 'general_category', 'brand']
num_fields = ['price', 'relative_price', 'time_since_start', 'time_since_prev']

# Flatten to collect uniques
from collections import Counter, defaultdict
counters = {f: Counter() for f in cat_fields}
sess_lengths = []

for ev_list in df['events_parsed']:
    sess_lengths.append(len(ev_list))
    for ev in ev_list:
        for f in cat_fields:
            val = ev.get(f, 'unknown')
            # unify missing / weird strings
            if val is None or (isinstance(val, float) and np.isnan(val)):
                val = 'unknown'
            counters[f][str(val)] += 1

# Show top uniques
for f in cat_fields:
    print(f, "unique:", len(counters[f]), "top:", counters[f].most_common(5))

# decide vocab sizes (example)
VOCAB_SIZES = {
    'event_type': None,       # leave full if small
    'category_code': 2000,    # keep top-2000 or so
    'general_category': None,
    'brand': 5000
}

def build_vocab(counter, max_size=None, min_freq=1, add_oov=True):
    items = [k for k, v in counter.items() if v >= min_freq]
    items_sorted = sorted(items, key=lambda x: counter[x], reverse=True)
    if max_size:
        items_sorted = items_sorted[:max_size]
    vocab = {v: i+1 for i, v in enumerate(items_sorted)}  # reserve 0 for PAD
    if add_oov:
        vocab['<OOV>'] = len(vocab) + 1
    return vocab

vocab = {}
for f in cat_fields:
    vocab[f] = build_vocab(counters[f], max_size=VOCAB_SIZES.get(f, None))
    print(f, 'vocab_size:', len(vocab[f]))

event_type unique: 2 top: [('2', 203322), ('0', 2057)]
category_code unique: 123 top: [('unknown.', 69033), ('electronics.smartphone', 56256), ('electronics.clocks', 6597), ('computers.notebook', 6076), ('electronics.video.tv', 4733)]
general_category unique: 14 top: [('electronics', 78013), ('unknown', 69033), ('appliances', 21564), ('computers', 12425), ('furniture', 6704)]
brand unique: 1859 top: [('unknown', 31276), ('samsung', 24639), ('xiaomi', 18236), ('apple', 15358), ('huawei', 5824)]
event_type vocab_size: 3
category_code vocab_size: 124
general_category vocab_size: 15
brand vocab_size: 1860


In [22]:
# 3) Feature encoding utilities
MAX_SEQ_LEN = 128  # tune after checking sess_lengths distribution
def map_cat(value, vocab_map):
    if value is None:
        return vocab_map.get('<OOV>', 1)
    s = str(value)
    if s in vocab_map:
        return vocab_map[s]
    else:
        return vocab_map.get('<OOV>', 1)

# prepare numeric scaler: compute from flattened events
all_nums = {f: [] for f in num_fields}
for ev_list in df['events_parsed']:
    for ev in ev_list:
        for nf in num_fields:
            val = ev.get(nf, 0.0)
            if val is None:
                val = 0.0
            all_nums[nf].append(float(val))

scalers = {}
for nf in num_fields:
    scaler = StandardScaler()
    vals = np.array(all_nums[nf]).reshape(-1, 1)
    scaler.fit(vals)
    scalers[nf] = scaler

In [23]:
# 4) Convert each session to fixed tensors (padded/truncated)
def session_to_features(ev_list):
    seq_len = min(len(ev_list), MAX_SEQ_LEN)
    cat_indices = {f: np.zeros(MAX_SEQ_LEN, dtype=np.int32) for f in cat_fields}
    num_values = np.zeros((MAX_SEQ_LEN, len(num_fields)), dtype=np.float32)
    mask = np.zeros((MAX_SEQ_LEN,), dtype=np.float32)
    for i in range(seq_len):
        ev = ev_list[i]
        mask[i] = 1.0
        for f in cat_fields:
            cat_indices[f][i] = map_cat(ev.get(f, 'unknown'), vocab[f])
        # numeric
        for j, nf in enumerate(num_fields):
            val = ev.get(nf, 0.0)
            if val is None:
                val = 0.0
            # scale
            scaled = scalers[nf].transform(np.array([[float(val)]]))[0,0]
            num_values[i, j] = scaled
    return cat_indices, num_values, mask

# Example transform on first few sessions
sample = df['events_parsed'].iloc[0]
cat_idx, num_vals, mask = session_to_features(sample)
print({k: v.shape for k, v in cat_idx.items()}, num_vals.shape, mask.sum())

{'event_type': (128,), 'category_code': (128,), 'general_category': (128,), 'brand': (128,)} (128, 4) 1.0


In [24]:
# 5) Build tf.data pipeline
def gen():
    for _, row in df.iterrows():
        cat_idx, num_vals, mask = session_to_features(row['events_parsed'])
        # combine categorical indices into a dict or stacked array
        # We'll return a dict for clarity
        features = {f: cat_idx[f] for f in cat_fields}
        features['numeric'] = num_vals
        features['mask'] = mask
        label = np.array([row['buy']], dtype=np.float32)
        yield features, label

# define output signature for tf.data
output_signature = (
    {
        **{f: tf.TensorSpec(shape=(MAX_SEQ_LEN,), dtype=tf.int32) for f in cat_fields},
        'numeric': tf.TensorSpec(shape=(MAX_SEQ_LEN, len(num_fields)), dtype=tf.float32),
        'mask': tf.TensorSpec(shape=(MAX_SEQ_LEN,), dtype=tf.float32)
    },
    tf.TensorSpec(shape=(1,), dtype=tf.float32)
)

dataset = tf.data.Dataset.from_generator(gen, output_signature=output_signature)
dataset = dataset.shuffle(2048).batch(64).prefetch(tf.data.AUTOTUNE)

# split into train/val manually (for simplicity)
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['buy'])
def make_ds_from_df(df_sub):
    def gen_sub():
        for _, row in df_sub.iterrows():
            cat_idx, num_vals, mask = session_to_features(row['events_parsed'])
            features = {f: cat_idx[f] for f in cat_fields}
            features['numeric'] = num_vals
            features['mask'] = mask
            label = np.array([row['buy']], dtype=np.float32)
            yield features, label
    return tf.data.Dataset.from_generator(gen_sub, output_signature=output_signature).batch(64).prefetch(tf.data.AUTOTUNE)

train_ds = make_ds_from_df(train_df)
val_ds = make_ds_from_df(val_df)

In [27]:
# 6) Build Transformer-based model in Keras
# helper: create embedding layers for each categorical feature
EMBED_DIM = 32
cat_emb_layers = {}
for f in cat_fields:
    vocab_size = len(vocab[f]) + 1  # plus pad maybe
    cat_emb_layers[f] = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=EMBED_DIM, mask_zero=True, name=f'emb_{f}')

# positional encoding (simple learned)
pos_emb = tf.keras.layers.Embedding(input_dim=MAX_SEQ_LEN, output_dim=EMBED_DIM, name='pos_emb')

class TransformerEncoderBlock(tf.keras.layers.Layer):
    def __init__(self, dim, num_heads, mlp_dim, dropout=0.1):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=dim)
        self.norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(dropout)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(mlp_dim, activation='relu'),
            tf.keras.layers.Dense(dim),
        ])
        self.norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout2 = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask=None, training=False):
        # mask: [batch, seq_len] float mask -> convert to attention mask
        attn_mask = None
        if mask is not None:
            # MultiHeadAttention takes boolean mask where True means keep (opposite behavior), or attention_mask with shape [batch, seq_len]?
            # We'll create a 2D key mask: shape (batch, seq_len)
            # Keras MultiHeadAttention supports attention_mask (boolean) shape [batch, seq_len]
            attn_mask = tf.cast(mask[:, tf.newaxis, :], dtype=tf.bool)  # shape (batch,1,seq)
        attn_out = self.mha(x, x, attention_mask=attn_mask)
        attn_out = self.dropout1(attn_out, training=training)
        out1 = self.norm1(x + attn_out)
        ff = self.ffn(out1)
        ff = self.dropout2(ff, training=training)
        out2 = self.norm2(out1 + ff)
        return out2

def build_model():
    # inputs
    inputs = {f: tf.keras.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name=f) for f in cat_fields}
    inputs['numeric'] = tf.keras.Input(shape=(MAX_SEQ_LEN, len(num_fields)), dtype=tf.float32, name='numeric')
    inputs['mask'] = tf.keras.Input(shape=(MAX_SEQ_LEN,), dtype=tf.float32, name='mask')

    # embed categorical and sum embeddings (or concat)
    emb_list = []
    for f in cat_fields:
        e = cat_emb_layers[f](inputs[f])  # (batch, seq, emb)
        emb_list.append(e)
    # project numeric features to embedding space
    numeric_proj = tf.keras.layers.Dense(EMBED_DIM)(inputs['numeric'])  # (batch, seq, emb)
    emb_list.append(numeric_proj)

    x = tf.keras.layers.Add()(emb_list)  # element-wise sum of embeddings
    # add position embedding
    positions = tf.range(start=0, limit=MAX_SEQ_LEN, delta=1)
    pos_embeddings = pos_emb(positions)
    x = x + pos_embeddings  # broadcast adds position to every batch

    # apply mask (pass to encoder blocks)
    mask = inputs['mask']

    # stack Transformer encoder blocks
    for _ in range(2):  # num encoder layers, tune as needed
        x = TransformerEncoderBlock(dim=EMBED_DIM, num_heads=4, mlp_dim=EMBED_DIM*4)(x, mask=mask)

    # pooling: use masked mean pooling
    mask_expanded = tf.keras.layers.Lambda(lambda m: tf.expand_dims(m, axis=-1))(mask)  # (batch, seq, 1)
    x_masked = tf.keras.layers.Multiply()([x, mask_expanded])
    sum_x = tf.keras.layers.Lambda(lambda z: tf.reduce_sum(z, axis=1))(x_masked)  # (batch, emb)
    lengths = tf.keras.layers.Lambda(lambda m: tf.reduce_sum(m, axis=1))(mask_expanded)  # (batch,1)
    pooled = tf.keras.layers.Lambda(lambda z: z[0] / (z[1] + 1e-6))([sum_x, lengths])


    # classifier head
    h = tf.keras.layers.Dense(64, activation='relu')(pooled)
    h = tf.keras.layers.Dropout(0.3)(h)
    out = tf.keras.layers.Dense(1, activation='sigmoid', name='buy_prob')(h)

    model = tf.keras.Model(inputs=inputs, outputs=out)
    return model

model = build_model()
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC(name='auc'), tf.keras.metrics.BinaryAccuracy(name='acc')])
model.summary()

c:\git_files\e-commerce_cohort\ecommerce\Lib\site-packages\keras\src\layers\layer.py:970: UserWarning: Layer 'transformer_encoder_block_2' (of type TransformerEncoderBlock) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\git_files\e-commerce_cohort\ecommerce\Lib\site-packages\keras\src\layers\layer.py:970: UserWarning: Layer 'transformer_encoder_block_3' (of type TransformerEncoderBlock) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ event_type          │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ category_code       │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ general_category    │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ brand (InputLayer)  │ (None, 128)       │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ numeric             │ (None, 128, 4)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_event_type      │ (None, 128, 32)   │        128 │ event_type[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_category_code   │ (None, 128, 32)   │      4,000 │ category_code[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_general_catego… │ (None, 128, 32)   │        512 │ general_category… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_brand           │ (None, 128, 32)   │     59,552 │ brand[0][0]       │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128, 32)   │        160 │ numeric[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 128, 32)   │          0 │ emb_event_type[0… │
│                     │                   │            │ emb_category_cod… │
│                     │                   │            │ emb_general_cate… │
│                     │                   │            │ emb_brand[0][0],  │
│                     │                   │            │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 128, 32)   │          0 │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mask (InputLayer)   │ (None, 128)       │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, 128, 32)   │     25,280 │ add_3[0][0],      │
│ (TransformerEncode… │                   │            │ mask[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, 128, 32)   │     25,280 │ transformer_enco… │
│ (TransformerEncode… │                   │            │ mask[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 128, 1)    │          0 │ mask[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 128, 32)   │          0 │ transformer_enco… │
│                     │                   │            │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 117,089 (457.38 KB)

 Trainable params: 117,089 (457.38 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 7) Train
# Option: compute class weights if imbalanced
labels = df['buy'].values
pos = labels.sum()
neg = len(labels) - pos
class_weight = {0: 1.0, 1: (neg / (pos + 1e-6))}  # simple balancing

# fit
history = model.fit(train_ds, validation_data=val_ds, epochs=10, class_weight=class_weight)

Epoch 1/10
    598/Unknown 175s 285ms/step - acc: 0.5845 - auc: 0.5690 - loss: 1.3211

c:\git_files\e-commerce_cohort\ecommerce\Lib\site-packages\keras\src\trainers\epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()
c:\git_files\e-commerce_cohort\ecommerce\Lib\site-packages\keras\src\layers\layer.py:970: UserWarning: Layer 'transformer_encoder_block_2' (of type TransformerEncoderBlock) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\git_files\e-commerce_cohort\ecommerce\Lib\site-packages\keras\src\layers\layer.py:970: UserWarning: Layer 'transformer_encoder_block_3' (of type TransformerEncoderBlock) was passed an input with a mask attached to it. However, this layer does not support ma

598/598 ━━━━━━━━━━━━━━━━━━━━ 189s 308ms/step - acc: 0.6076 - auc: 0.6134 - loss: 1.3132 - val_acc: 0.6791 - val_auc: 0.7263 - val_loss: 0.6408
Epoch 2/10


c:\git_files\e-commerce_cohort\ecommerce\Lib\site-packages\keras\src\trainers\epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


598/598 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step - acc: 0.7344 - auc: 0.7387 - loss: 1.1252

...

c:\git_files\e-commerce_cohort\ecommerce\Lib\site-packages\keras\src\trainers\epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


598/598 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - acc: 0.8422 - auc: 0.8860 - loss: 0.7887

c:\git_files\e-commerce_cohort\ecommerce\Lib\site-packages\keras\src\trainers\epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


598/598 ━━━━━━━━━━━━━━━━━━━━ 137s 228ms/step - acc: 0.8401 - auc: 0.8868 - loss: 0.8100 - val_acc: 0.8217 - val_auc: 0.7740 - val_loss: 0.4399


c:\git_files\e-commerce_cohort\ecommerce\Lib\site-packages\keras\src\trainers\epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


In [29]:
# 8) Evaluate and inference
# example inference on raw session
def predict_session(model, ev_list):
    cat_idx, num_vals, mask = session_to_features(ev_list)
    feed = {f: np.expand_dims(cat_idx[f], 0) for f in cat_fields}
    feed['numeric'] = np.expand_dims(num_vals, 0)
    feed['mask'] = np.expand_dims(mask, 0)
    prob = model.predict(feed)[0,0]
    return prob

In [34]:
example_session = [
    {
        "event_type": 2, 
        "product_id": 22200103, 
        "category_code": "electronics.smartphone",
        "general_category": "electronics",
        "brand": "samsung",
        "price": 350.0,
        "relative_price": 0.45,
        "time_since_start": 0.0,
        "time_since_prev": 0.0
    },
    {
        "event_type": 2, 
        "product_id": 6902133, 
        "category_code": "computers.notebook",
        "general_category": "computers",
        "brand": "apple",
        "price": 1200.0,
        "relative_price": 0.9,
        "time_since_start": 30.0,
        "time_since_prev": 30.0
    }
]

prob = predict_session(model, example_session)
print(f"구매 확률: {prob:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
구매 확률: 0.4306


In [56]:
example_session = [
    {
        "event_type": 2, 
        "product_id": 22200103, 
        "category_code": "electronics.smartphone",
        "general_category": "electronics",
        "brand": "samsung",
        "price": 350.0,
        "relative_price": 0.45,
        "time_since_start": 0.0,
        "time_since_prev": 0.0
    },
    {
        "event_type": 2, 
        "product_id": 6902133, 
        "category_code": "computers.notebook",
        "general_category": "computers",
        "brand": "apple",
        "price": 1200.0,
        "relative_price": 0.9,
        "time_since_start": 14.0,
        "time_since_prev": 14.0
    },
    {
        "event_type": 2, 
        "product_id": 22200103, 
        "category_code": "electronics.smartphone",
        "general_category": "electronics",
        "brand": "samsung",
        "price": 350.0,
        "relative_price": 0.45,
        "time_since_start": 27.0,
        "time_since_prev": 13.0
    },
    {
        "event_type": 1, 
        "product_id": 22200103, 
        "category_code": "electronics.smartphone",
        "general_category": "electronics",
        "brand": "samsung",
        "price": 350.0,
        "relative_price": 0.45,
        "time_since_start": 28.0,
        "time_since_prev": 1.0
    }
]

prob = predict_session(model, example_session)
print(f"구매 확률: {prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
구매 확률: 0.5014


In [58]:
example_session = [
    {
        "event_type": 2, 
        "product_id": 22200103, 
        "category_code": "electronics.smartphone",
        "general_category": "electronics",
        "brand": "samsung",
        "price": 350.0,
        "relative_price": 0.45,
        "time_since_start": 0.0,
        "time_since_prev": 0.0
    },
    {
        "event_type": 2, 
        "product_id": 6902133, 
        "category_code": "computers.notebook",
        "general_category": "computers",
        "brand": "apple",
        "price": 1200.0,
        "relative_price": 0.9,
        "time_since_start": 14.0,
        "time_since_prev": 14.0
    },
    {
        "event_type": 2, 
        "product_id": 22200103, 
        "category_code": "electronics.smartphone",
        "general_category": "electronics",
        "brand": "samsung",
        "price": 350.0,
        "relative_price": 0.45,
        "time_since_start": 27.0,
        "time_since_prev": 13.0
    }
]

prob = predict_session(model, example_session)
print(f"구매 확률: {prob:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
구매 확률: 0.3777


In [53]:
print(log_dict.iloc[7]['event'])

[{'event_type': np.int64(2), 'product_id': 17301541, 'category_code': 'unknown.', 'general_category': 'unknown', 'brand': 'unknown', 'price': 162.17, 'relative_price': 1.8000888000887998, 'time_since_start': 0.0, 'time_since_prev': 0}, {'event_type': np.int64(2), 'product_id': 17301541, 'category_code': 'unknown.', 'general_category': 'unknown', 'brand': 'unknown', 'price': 162.17, 'relative_price': 1.8000888000887998, 'time_since_start': 25.0, 'time_since_prev': 25.0}]


In [52]:
prob = predict_session(model, log_dict.iloc[7]['event'])
print(f"구매 확률: {prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
구매 확률: 0.5479


In [54]:
len(log_dict)

42509

In [59]:
import random
randlist = random.sample(range(len(log_dict)), 5)
randlist

[30033, 9564, 37938, 18471, 17924]

In [62]:
for i  in randlist:
    prob = predict_session(model, log_dict.iloc[i]['event'])
    print(i)
    print(f"Index {i} 구매 확률: {prob:.4f}")
    print("-"*42)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
30033
Index 30033 구매 확률: 0.3113
------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
9564
Index 9564 구매 확률: 0.2456
------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
37938
Index 37938 구매 확률: 0.2646
------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
18471
Index 18471 구매 확률: 0.1179
------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
17924
Index 17924 구매 확률: 0.0370
------------------------------------------


In [65]:
print(log_dict.iloc[17924]['event'])

[{'event_type': np.int64(2), 'product_id': 21407370, 'category_code': 'electronics.clocks', 'general_category': 'electronics', 'brand': 'casio', 'price': 162.17, 'relative_price': 0.642130271233419, 'time_since_start': 0.0, 'time_since_prev': 0}, {'event_type': np.int64(2), 'product_id': 21400403, 'category_code': 'electronics.clocks', 'general_category': 'electronics', 'brand': 'boccia', 'price': 87.52, 'relative_price': 0.34654523856662045, 'time_since_start': 312.0, 'time_since_prev': 312.0}]
